# 3rd Level Model Structure: Single Stage Reactive Distillation

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))

import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
from copy import deepcopy

In [2]:
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule
from stages.non_reactive_stage import non_reactive_stage_rule

from utility.display_utility import trans_product_mole, trans_product_mass, beautify, beautify2
from utility.model_utility import add_dual, update_dual, check_DOF
from utility.data_utility import cal_cnumber

model = pe.ConcreteModel()

# Global Set

In [3]:
model.TRAY = pe.RangeSet(1,3)

# Construct Reactive Stages

In [4]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].T_F
| reactive[1].P
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].T
| reactive[1].H_F
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].r_total_comp
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_block.r_WGS
| reactive[1].kinetics_bloc

# Construct a single condenser

In [5]:
model.condenser = pe.Block(rule=condenser_stage_rule)

| Importing Condenser Stage......
| Adding the following local variable:
------------------------------------
| condenser.T
| condenser.T_F
| condenser.P
| condenser.Q_main
| condenser.x_
| condenser.y_
| condenser.x
| condenser.y
| condenser.z
| condenser.L
| condenser.W
| condenser.V
| condenser.F
| condenser.H_L_
| condenser.H_V_
| condenser.H_L
| condenser.H_V
| condenser.H_F
| condenser.f_V
| condenser.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.energy_block.dH_F
| condenser.energy_block.dH_V
| condenser.energy_block.dH_L
| condenser.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.VLE_block.n_ave
| condenser.VLE_block.n_ave_cal
| condenser.VLE_block.Hen
| condenser.VLE_block.Hen0
| condenser.VLE_blo

# Construct a single reboiler - pf

In [6]:
model.reboiler = pe.Block(rule=non_reactive_stage_rule)

> Importing Non Reactive Stage......
> Adding the following local variable:
------------------------------------
| reboiler.T_F
| reboiler.P
| reboiler.Q_main
| reboiler.x_
| reboiler.y_
| reboiler.x
| reboiler.y
| reboiler.z
| reboiler.L
| reboiler.V
| reboiler.F
| reboiler.H_L_
| reboiler.H_V_
| reboiler.H_L
| reboiler.H_V
| reboiler.T
| reboiler.H_F
| reboiler.f_V
| reboiler.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| reboiler.energy_block.dH_F
| reboiler.energy_block.dH_V
| reboiler.energy_block.dH_L
| reboiler.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| reboiler.VLE_block.P_VLE
| reboiler.VLE_block.n_ave
| reboiler.VLE_block.n_ave_cal
| reboiler.VLE_block.Hen
| reboiler.VLE_block.Hen0
| reboiler.VLE_block.gamma
| r

# Linking Stage Variables

### Vapor Between Reactive Stages

In [7]:
def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

### Liquid Between Reactive Stages

In [8]:
def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

### Condenser Vapor

In [9]:
def V_condenser_rule(model):
    return model.reactive[model.TRAY.first()].V['out'] == model.condenser.V['in']
model.V_condenser_con = pe.Constraint(rule=V_condenser_rule)

def Vy_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].y[i] == model.condenser.y_['in',i]
model.Vy_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_condenser_rule)

def Vh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_V == model.condenser.H_V_['in']
model.Vh_condenser_con = pe.Constraint(rule=Vh_condenser_rule)

### Condenser Liquid

In [10]:
def L_condenser_rule(model):
    return model.reactive[model.TRAY.first()].L['in'] == model.condenser.L['out']
model.L_condenser_con = pe.Constraint(rule=L_condenser_rule)

def Lx_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].x_['in',i] == model.condenser.x[i]
model.Lx_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_condenser_rule)

def Lh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_L_['in'] == model.condenser.H_L
model.Lh_condenser_con = pe.Constraint(rule=Lh_condenser_rule)

### Reboiler Vapor

In [11]:
def V_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].V['in'] == model.reboiler.V['out']
model.V_reboiler_con = pe.Constraint(rule=V_reboiler_rule)

def Vy_reboiler_rule(model,i):
    return model.reactive[model.TRAY.last()].y_['in',i] == model.reboiler.y[i]
model.Vy_reboiler_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_reboiler_rule)

def Vh_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].H_V_['in'] == model.reboiler.H_V
model.Vh_reboiler_con = pe.Constraint(rule=Vh_reboiler_rule)

### Reboiler Liquid

In [12]:
def L_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].L['out'] == model.reboiler.L['in']
model.L_reboiler_con = pe.Constraint(rule=L_reboiler_rule)

def Lx_reboiler_rule(model,i):
    return model.reactive[model.TRAY.last()].x[i] == model.reboiler.x_['in',i]
model.Lx_reboiler_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_reboiler_rule)

def Lh_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].H_L == model.reboiler.H_L_['in']
model.Lh_reboiler_con = pe.Constraint(rule=Lh_reboiler_rule)

In [13]:
model.obj = pe.Objective(expr = sum(model.reactive[j].T - model.reactive[j].MPCC.pf for j in model.TRAY)\
                         - model.reboiler.MPCC.pf ,sense=pe.maximize)
# model.obj = pe.Objective(expr = sum(model.reactive[j].T - model.reactive[j].MPCC.pf for j in model.TRAY) ,sense=pe.maximize)
# model.obj = pe.Objective(expr = sum(model.reactive[j].T for j in model.TRAY) ,sense=pe.maximize)

In [14]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


# Load from Reactive Flash solutions

In [15]:
with open('../saved_solutions/reactive_flash_MPCC_P_pf_200C.pickle', 'rb') as f:
    results_imported = pickle.load(f)
results_changed = deepcopy(results_imported)

### Duplicate variable solution and bounds multiplier

In [16]:
results_changed.Solution.Variable = {}
for i in list(results_imported.Solution.Variable.keys()):
    for j in model.TRAY:
        results_changed.Solution.Variable['reactive[{}].{}'.format(j,i)] = \
        results_imported.Solution.Variable[i]

### Duplicate constraint multiplier

In [17]:
results_changed.Solution.Constraint = {}
for i in list(results_imported.Solution.Constraint.keys()):
    for j in model.TRAY:
        results_changed.Solution.Constraint['reactive[{}].{}'.format(j,i)] = \
        results_imported.Solution.Constraint[i]

# Load from Condenser solutions

In [18]:
with open('../saved_solutions/condenser_MPCC_P_pf_30C.pickle', 'rb') as f:
    condenser_results_imported = pickle.load(f)

### Duplicate variable solution and bounds multiplier

In [19]:
for i in list(condenser_results_imported.Solution.Variable.keys()):
    results_changed.Solution.Variable['condenser.{}'.format(i)] = \
    condenser_results_imported.Solution.Variable[i]

### Duplicate constraint multiplier

In [20]:
for i in list(condenser_results_imported.Solution.Constraint.keys()):
    results_changed.Solution.Constraint['condenser.{}'.format(i)] = \
    condenser_results_imported.Solution.Constraint[i]

# Load from Reboiler solutions

In [21]:
with open('../saved_solutions/reboiler_MPCC_P_pf_200C.pickle', 'rb') as f:
    reboiler_results_imported = pickle.load(f)

### Duplicate variable solution and bounds multiplier

In [22]:
for i in list(reboiler_results_imported.Solution.Variable.keys()):
    results_changed.Solution.Variable['reboiler.{}'.format(i)] = \
    reboiler_results_imported.Solution.Variable[i]

### Duplicate constraint multiplier

In [23]:
for i in list(reboiler_results_imported.Solution.Constraint.keys()):
    results_changed.Solution.Constraint['reboiler.{}'.format(i)] = \
    reboiler_results_imported.Solution.Constraint[i]

In [24]:
model.solutions.load_from(results_changed)

# Fixing Redundent Stream Variables

In [25]:
# condenser
model.condenser.VLE_block.n_ave.fix(4)

model.condenser.F.fix(0)
model.condenser.T_F.fix(300+273.15)
model.condenser.z.fix(0)

model.condenser.V['P'].fix(0)
model.condenser.L['in'].fix(0)
for i in m.COMP_TOTAL: model.condenser.x_['in',i].fix(0)
model.condenser.H_L_['in'].fix(0)

In [26]:
# reboiler
model.reboiler.VLE_block.n_ave.fix(20)

model.reboiler.F.fix(0)
model.reboiler.T_F.fix(300+273.15)
model.reboiler.z.fix(0)

model.reboiler.V['P'].fix(0)
model.reboiler.V['in'].fix(0)
for i in m.COMP_TOTAL: model.reboiler.y_['in',i].fix(0)
model.reboiler.H_V_['in'].fix(0)
model.reboiler.L['P'].fix(0)

# Load Operating Parameters

In [27]:
# condenser
model.condenser.P.fix(19)
# model.condenser.T.fix(30+273.15)
model.condenser.T.fix(30+273.15)
model.condenser.L['out'].fix(0)

# reboiler
model.reboiler.P.fix(20)
# model.reboiler.T.fix(30+273.15)
model.reboiler.T.fix(200+273.15)
# model.reboiler.Q_main.fix(0)

# reactive stage
for j in model.reactive:
    model.reactive[j].cat.fix(5000)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(20)
    
    model.reactive[j].F.fix(1)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(1/(1+2)-0/2)
    model.reactive[j].z['H2'].fix(2/(1+2)-0/2)
    model.reactive[j].z['C30H62'].fix(0)
    model.reactive[j].V['P'].fix(0)
    model.reactive[j].L['P'].fix(0)
    # model.reactive[j].Q_main.fix(0)
    model.reactive[j].T.setub(200+273.15)

In [28]:
check_DOF(pe,model)

Active Equality Constraints:	 6108
Active Inequality Constraints:	 4
Active Variables:		 6335
Fixed Variables:		 212
DOF:				 15


In [29]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'

opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'

opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6
# opt.options['recalc_y'] = 'yes'
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
warm_start_init_point=yes
warm_start_bound_push=1e-20
warm_start_mult_bound_push=1e-20
mu_init=1e-06
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                                 mu_init = 1e-06                 yes
                      print_user_options = yes                   yes
                   warm_start_bound_push = 1e-20                 yes
                   warm_start_init_point = yes                   yes
              warm_start_mult_bound_push = 1e-20                 yes

****************************************************

  68r-1.4161430e+03 2.28e+02 9.99e+02   1.1 0.00e+00    -  0.00e+00 1.22e-10R  2
  69r-1.4161435e+03 2.28e+02 9.99e+02   1.1 2.91e+07    -  1.28e-08 1.89e-09f  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70r-1.4161435e+03 2.28e+02 1.05e+03   1.1 5.07e+03    -  6.75e-03 2.85e-08f  2
  71r-1.4159203e+03 2.24e+02 1.97e+03   0.4 2.46e+03    -  2.61e-02 5.12e-03f  1
  72 -1.4159206e+03 2.24e+02 1.66e+01  -6.0 1.14e+03    -  9.81e-04 5.58e-05h  1
  73 -1.4159234e+03 2.24e+02 1.45e+03  -6.0 8.45e+03    -  8.24e-04 1.11e-05h  1
  74 -1.4159526e+03 2.24e+02 2.58e+03  -6.0 8.08e+03    -  5.36e-04 1.04e-04h  1
  75r-1.4159526e+03 2.24e+02 9.99e+02   0.5 0.00e+00    -  0.00e+00 6.03e-11R  2
  76r-1.4159530e+03 2.24e+02 2.24e+04   0.5 2.31e+06    -  8.08e-07 1.91e-08f  1
  77r-1.4159529e+03 2.24e+02 2.22e+04   0.5 2.89e+03    -  9.87e-03 1.73e-06f  1
  78r-1.4157956e+03 2.22e+02 2.20e+04   0.5 4.72e+02    -  5.94e-03 6.50e-03f  1
  79 -1.4158000e+03 2.22e+02

 161 -1.4143395e+03 9.99e+00 1.25e+04  -6.0 1.09e+02  -3.0 1.86e-02 2.15e-02f  1
 162 -1.4149546e+03 9.54e+00 1.19e+04  -6.0 1.28e+02  -3.5 2.36e-02 4.99e-02f  1
 163 -1.4156758e+03 9.32e+00 1.14e+05  -6.0 7.68e+01  -3.0 1.10e-02 2.41e-01h  1
 164 -1.4156758e+03 9.32e+00 2.58e+06  -6.0 2.11e+02  -3.5 6.85e-02 7.25e-07h  2
 165 -1.4163085e+03 8.61e+00 2.03e+06  -6.0 2.04e+02    -  1.07e-06 9.38e-02f  1
 166 -1.4164598e+03 7.33e+00 1.47e+06  -6.0 2.26e+02    -  5.92e-02 2.09e-01H  1
 167 -1.4163730e+03 7.21e+00 1.46e+07  -6.0 8.14e+01  -3.1 1.52e-06 1.63e-02h  5
 168 -1.4155734e+03 6.47e+00 3.40e+08  -6.0 8.19e+01  -2.7 4.72e-04 1.63e-01h  2
 169 -1.4154439e+03 1.60e+01 6.41e+07  -6.0 1.05e+02  -3.1 4.33e-02 9.86e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170 -1.4154474e+03 1.60e+01 6.41e+07  -6.0 2.67e+02  -3.6 7.72e-02 2.26e-04h  1
 171 -1.4159389e+03 1.41e+01 2.09e+08  -6.0 1.23e+02  -3.2 4.45e-05 1.23e-01f  1
 172 -1.4169498e+03 1.31e+01

In [30]:
model.reboiler.T.fix(350+273.15)

In [31]:
results = opt.solve(model,tee=False)
update_dual(pe,model)

In [32]:
model.reactive[1].T.setub(205+273.15)
model.reactive[2].T.setub(210+273.15)
model.reactive[3].T.setub(215+273.15)

In [33]:
results = opt.solve(model,tee=False)
update_dual(pe,model)

In [34]:
beautify(pe,model)

Here comes the result:
------------------------------------------------------------------------------------------------------------
stages		T		Q		V_out		L_out		L_P		W
Condenser	30.00		-40.38		0.65338252	0.00000000	0.02576675	0.68366935

stages		T		Q		V_out		L_out		L_P		P_VLE
Reactive[1]	205.00		-43.68		1.36281862	0.01016793	0.00000000	20.00000050
Reactive[2]	210.00		-45.14		0.88992550	0.02021897	0.00000000	20.00000018
Reactive[3]	215.00		-46.02		0.43905081	0.02975881	0.00000000	20.00000007
Reboiler	350.00		3.23		0.00282247	0.02693634			19.99999957
------------------------------------------------------------------------------------------------------------
		Condenser:	Vapor		Liquid		Reboiler	Vapor		Liquid
H2 				68.609%		0.903%				7.168%		0.403%
CO 				6.599%		0.186%				0.473%		0.029%
CO2 				20.437%		3.971%				6.073%		0.578%
H2O 				0.223%		0.000%				68.628%		8.463%
C2H4 				0.618%		0.193%				0.300%		0.034%
C3H6 				0.653%		1.092%				0.525%		0.100%
C4H8 				0.495%		2.676%				0.554%		0.

# So, what exactly does adding a reflux do?

In [35]:
# opt.options['warm_start_init_point'] = 'yes'
# opt.options['warm_start_bound_push'] = 1e-20
# opt.options['warm_start_mult_bound_push'] = 1e-20
# opt.options['mu_init'] = 1e-6

In [36]:
results_anchor = results
update_dual(pe,model)
model.solutions.store_to(results_anchor)

In [37]:
Refluxrange = np.linspace(1,2,21)
Trange = np.linspace(240+273.15,270+273.15,15)

In [38]:
cd_data_master = {}
rf_data_master = {}

for t in Trange:
    model.del_component(model.obj)
    model.obj = pe.Objective(expr = sum(model.reactive[j].T - model.reactive[j].MPCC.pf for j in model.TRAY) - 100*model.reboiler.MPCC.pf ,sense=pe.maximize)
    model.solutions.load_from(results_anchor)
    update_dual(pe,model)
    
    for j in model.reactive:
        model.reactive[j].T.fixed = False
        model.reactive[j].T.setub(t+5*j)
    model.condenser.L['out'].fix(1)
    
    results_anchor = opt.solve(model,tee=True)
    model.solutions.store_to(results_anchor)
    
    model.del_component(model.obj)
    model.obj = pe.Objective(expr = model.condenser.L['out'] - sum(model.reactive[j].MPCC.pf for j in model.TRAY) - 100*model.reboiler.MPCC.pf,sense=pe.maximize)
    model.condenser.L['out'].fixed = False
    for j in model.reactive:
        model.reactive[j].T.fixed=True
    
    print('\nWorking on T = {} K'.format(model.reactive[1].T.value))
    print('-'*108)
    
    cd_data = {};
    cd_data['Re'] = []; cd_data['D'] = []; cd_data['V'] = []
    cd_data['x'] = {}; cd_data['y'] = {}; cd_data['g'] = {}; cd_data['d'] = {}; cd_data['b'] = {}
    for i in m.COMP_TOTAL:
        cd_data['x'][i] = []
        cd_data['y'][i] = []
        cd_data['g'][i] = []
        cd_data['d'][i] = []
        cd_data['b'][i] = []

    rf_data = {}
    for j in model.reactive:
        rf_data[j] = {}
        rf_data[j]['r'] = {}; rf_data[j]['b'] = {}; rf_data[j]['x'] = {};rf_data[j]['y'] = {};
        rf_data[j]['T'] = []; rf_data[j]['Q'] = []; rf_data[j]['V'] = []; rf_data[j]['L'] = []; 
        rf_data[j]['r_WGS'] = []; rf_data[j]['r_FT'] = []; rf_data[j]['pf'] = []
        for i in m.COMP_TOTAL:
            rf_data[j]['r'][i] = []
            rf_data[j]['b'][i] = []
            rf_data[j]['x'][i] = []
            rf_data[j]['y'][i] = []       

    for re in Refluxrange:
        model.condenser.L['out'].setub(re)
        results = opt.solve(model,tee=True)
        update_dual(pe,model)
        print('Solved\t|Reflux = {:.3f} kmol/s\t|Vapor = {:.3f} kmol/s\t|Distillate = {:.3f} kmol/s\t|Bottom = {:.3f} kmol/s'.\
              format(model.condenser.L['out'].value,model.condenser.V['out'].value,model.condenser.L['P'].value,model.reactive[3].L['out'].value))

        cd_data['V'].append(model.condenser.V['out'].value)
        cd_data['D'].append(model.condenser.L['P'].value)
        cd_data['Re'].append(model.condenser.L['out'].value)

        for i in model.reactive[1].r_total_comp:
            cd_data['x'][i].append(model.condenser.x[i].value)
            cd_data['y'][i].append(model.condenser.y[i].value)
            cd_data['g'][i].append(model.condenser.y[i].value*model.condenser.V['out'].value)
            cd_data['d'][i].append(model.condenser.x[i].value*model.condenser.L['P'].value)
            cd_data['b'][i].append(model.condenser.x[i].value*model.condenser.L['out'].value)

        for j in model.reactive:      
            rf_data[j]['T'].append(model.reactive[j].T.value)
            rf_data[j]['Q'].append(model.reactive[j].Q_main.value)
            rf_data[j]['V'].append(model.reactive[j].V['out'].value)
            rf_data[j]['L'].append(model.reactive[j].L['out'].value)
            rf_data[j]['r_WGS'].append(model.reactive[j].kinetics_block.r_WGS.value)
            rf_data[j]['r_FT'].append(model.reactive[j].kinetics_block.r_FT_total.value)
            rf_data[j]['pf'].append(model.reactive[j].MPCC.pf.value)

            for i in model.reactive[1].r_total_comp:
                rf_data[j]['r'][i].append(model.reactive[j].r_total_comp[i].value)
                rf_data[j]['b'][i].append(model.reactive[j].x[i].value*model.reactive[j].L['out'].value)
                rf_data[j]['x'][i].append(model.reactive[j].x[i].value)
                rf_data[j]['y'][i].append(model.reactive[j].y[i].value)
        
    cd_data_master[t] = cd_data
    rf_data_master[t] = rf_data

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
warm_start_init_point=yes
warm_start_bound_push=1e-20
warm_start_mult_bound_push=1e-20
mu_init=1e-06
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                                 mu_init = 1e-06                 yes
                      print_user_options = yes                   yes
                   warm_start_bound_push = 1e-20                 yes
                   warm_start_init_point = yes                   yes
              warm_start_mult_bound_push = 1e-20                 yes

****************************************************

  69 -1.5147314e+03 8.59e+00 1.34e+04  -6.0 1.36e+03    -  4.68e-01 3.41e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70 -1.5165613e+03 8.42e+00 1.28e+04  -6.0 2.32e+03    -  4.10e-01 4.63e-02f  1
  71 -1.5178572e+03 8.39e+00 6.31e+05  -6.0 3.45e+03    -  7.87e-02 3.26e-02f  1
  72 -1.5184131e+03 8.18e+00 6.46e+05  -6.0 1.59e+03    -  2.97e-02 2.63e-02h  1
  73 -1.5188409e+03 8.02e+00 3.05e+06  -6.0 1.65e+03    -  5.47e-01 1.95e-02h  1
  74 -1.5189842e+03 7.97e+00 2.45e+06  -6.0 1.69e+03    -  5.27e-02 6.55e-03h  1
  75 -1.5201309e+03 7.58e+00 6.61e+05  -6.0 1.71e+03    -  1.70e-01 5.26e-02f  1
  76 -1.5254769e+03 7.42e+00 3.50e+05  -6.0 1.82e+03    -  6.42e-02 2.92e-01f  1
  77 -1.5254769e+03 7.42e+00 4.17e+07  -6.0 2.47e+03    -  5.45e-01 9.86e-07h  1
In iteration 77, 1 Slack too small, adjusting variable bound
  78 -1.5254770e+03 7.42e+00 1.43e+07  -6.0 2.09e+03    -  2.33e-06 5.60e-06h  1
  79 -1.5254774e+03 7.42e+00 2.02e+07  -6.0 2.43


Working on T = 518.15 K
------------------------------------------------------------------------------------------------------------
Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
warm_start_init_point=yes
warm_start_bound_push=1e-20
warm_start_mult_bound_push=1e-20
mu_init=1e-06
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                                 mu_init = 1e-06                 yes
                      print_user_options = yes                   yes
                   warm_start_bound_push = 1e-20                 yes
                   warm_start_init_point = yes           

   5 -1.0499859e+00 2.30e-05 4.13e+00  -6.0 1.97e-04    -  1.00e+00 8.87e-01h  1
   6 -1.0499841e+00 1.91e-12 2.28e-04  -6.0 2.05e-05    -  1.00e+00 1.00e+00f  1
   7 -1.0499841e+00 2.27e-13 4.81e-06  -6.0 1.33e-06    -  1.00e+00 1.00e+00h  1
   8 -1.0500010e+00 1.23e-10 3.36e-01  -9.0 1.72e-04    -  1.00e+00 9.97e-01f  1
   9 -1.0500010e+00 2.08e-11 2.13e+00  -9.0 3.88e-07    -  1.00e+00 8.31e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10 -1.0500010e+00 1.96e-11 3.89e+01  -9.0 4.61e-06    -  1.00e+00 5.88e-02f  2
  11 -1.0500010e+00 2.27e-13 7.84e-08  -9.0 2.29e-07    -  1.00e+00 1.00e+00h  1
  12 -1.0500010e+00 1.71e-13 1.23e-10  -9.0 1.84e-08    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 12

                                   (scaled)                 (unscaled)
Objective...............:  -1.0500010337349015e+00   -1.0500010337349015e+00
Dual infeasibility......:   1.2337677433750386e-10    1.2337677433750386e-10
Constraint viol

   1 -1.1498757e+00 6.31e-02 6.68e+01  -6.0 2.99e+00    -  1.00e+00 9.98e-01h  1
   2 -1.1499587e+00 1.86e-02 1.97e+01  -6.0 5.41e-02    -  1.00e+00 7.05e-01h  1
   3 -1.1499764e+00 8.44e-03 8.94e+00  -6.0 1.57e-02    -  1.00e+00 5.47e-01f  1
   4 -1.1499869e+00 1.96e-04 2.07e-01  -6.0 7.21e-03    -  1.00e+00 9.77e-01f  1
   5 -1.1499860e+00 2.80e-05 5.10e+00  -6.0 1.60e-04    -  1.00e+00 8.57e-01h  1
   6 -1.1499842e+00 1.74e-12 3.17e-04  -6.0 2.19e-05    -  1.00e+00 1.00e+00f  1
   7 -1.1499841e+00 1.71e-13 1.09e-05  -6.0 1.96e-06    -  1.00e+00 1.00e+00h  1
   8 -1.1499841e+00 1.71e-13 3.32e-08  -6.0 2.12e-07    -  1.00e+00 1.00e+00h  1
   9 -1.1500010e+00 1.06e-10 3.59e-01  -9.0 1.49e-04    -  1.00e+00 9.96e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10 -1.1500010e+00 1.79e-11 2.19e+00  -9.0 4.02e-07    -  1.00e+00 8.31e-01f  1
  11 -1.1500010e+00 1.67e-11 4.18e+01  -9.0 3.62e-06    -  1.00e+00 6.98e-02f  2
  12 -1.1500010e+00 1.71e-13

...

2687 -1.6124428e+03 5.82e-01 1.00e+02  -9.0 4.38e+07    -  3.38e-06 1.18e-06f  1
2688 -1.6125799e+03 5.82e-01 1.00e+02  -9.0 8.25e+06    -  5.82e-06 3.80e-06f  2
2689 -1.6127367e+03 5.82e-01 1.00e+02  -9.0 9.74e+05    -  4.53e-06 4.71e-06f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2690 -1.6128366e+03 5.82e-01 1.00e+02  -9.0 9.19e+06    -  6.51e-11 2.99e-06f  1
2691 -1.6134767e+03 5.81e-01 1.00e+02  -9.0 5.77e+05    -  3.00e-05 2.03e-05f  3
2692 -1.6142682e+03 5.81e-01 1.00e+02  -9.0 4.88e+05    -  2.58e-04 3.13e-05f  1
2693 -1.6199307e+03 5.82e-01 9.99e+01  -9.0 2.54e+05    -  9.32e-04 4.98e-04f  1
2694 -1.6205541e+03 5.80e-01 9.98e+01  -9.0 2.75e+04    -  7.18e-03 1.67e-03f  1
2695 -1.6205619e+03 5.76e-01 9.89e+01  -9.0 1.07e+04    -  1.40e-02 8.82e-03h  1
2696 -1.6205626e+03 5.62e-01 9.66e+01  -9.0 1.03e+04    -  9.69e-03 2.27e-02h  1
2697 -1.6205631e+03 5.61e-01 1.40e+02  -9.0 1.14e+04    -  8.24e-02 2.14e-03h  1
2698 -1.6205703e+03 6.54e-01

2777 -1.6208753e+03 5.00e-05 1.80e+10  -9.0 1.52e+00    -  3.63e-01 7.43e-03h  1
2778 -1.6208754e+03 5.29e-05 1.54e+10  -9.0 2.61e+00    -  2.82e-01 1.45e-01f  1
2779 -1.6208759e+03 1.00e-03 4.51e+08  -9.0 3.69e+00    -  6.77e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2780 -1.6208763e+03 7.75e-03 1.87e+11  -9.0 1.05e+01    -  3.56e-01 1.00e+00h  1
2781 -1.6208766e+03 1.45e-02 1.04e+17  -9.0 1.51e+01    -  9.12e-01 1.00e+00h  1
2782 -1.6208779e+03 5.06e-01 2.13e+22  -9.0 9.36e+01    -  2.62e-01 1.00e+00h  1
2783 -1.6208783e+03 2.15e+00 5.86e+28  -9.0 1.82e+02    -  5.96e-01 1.00e+00h  1
2784 -1.6208782e+03 2.38e-01 4.78e+33  -9.0 4.72e+01    -  5.80e-01 1.00e+00h  1
Scaling factors are invalid - setting them all to 1.
2785r-1.6208782e+03 2.38e-01 9.99e+02  -0.6 0.00e+00  19.4 0.00e+00 0.00e+00R  1
2786r-1.6208781e+03 2.38e-01 9.99e+02  -0.6 5.00e+04    -  1.12e-07 2.87e-08f  1
2787r-1.6208778e+03 2.38e-01 9.99e+02  -0.6 1.03e+04    

2868 -1.6208576e+03 1.16e+00 8.78e+01  -9.0 7.86e+02    -  5.32e-02 7.55e-03h  4
2869 -1.6208598e+03 1.13e+00 8.51e+01  -9.0 4.02e+02    -  3.16e-02 2.83e-02h  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2870 -1.6208619e+03 1.09e+00 7.26e+01  -9.0 2.58e+02    -  1.51e-01 3.77e-02h  3
2871 -1.6208657e+03 9.56e-01 9.73e+01  -9.0 1.70e+02    -  7.36e-02 1.28e-01h  1
2872r-1.6208657e+03 9.56e-01 9.99e+02  -0.1 0.00e+00  18.4 0.00e+00 4.77e-07R 22
2873r-1.6208657e+03 9.56e-01 9.99e+02  -0.1 1.03e+06    -  4.56e-10 1.90e-09f  2
2874r-1.6208657e+03 9.56e-01 9.99e+02  -0.1 7.36e+03    -  5.85e-09 8.24e-09f  2
2875r-1.6208657e+03 9.56e-01 9.99e+02  -0.1 3.13e+03    -  2.69e-08 1.24e-08f  2
2876r-1.6208657e+03 9.56e-01 9.99e+02  -0.1 1.68e+03    -  7.76e-08 7.89e-08f  2
2877r-1.6208657e+03 9.56e-01 9.99e+02  -0.1 1.55e+03    -  3.21e-07 2.12e-07f  2
2878r-1.6208656e+03 9.54e-01 5.73e+03  -0.1 1.53e+03    -  7.19e-07 1.79e-06f  1
2879r-1.6208655e+03 9.51e-01

2962 -1.6068214e+03 1.06e-01 9.75e+01  -9.0 2.70e+02    -  8.15e-03 8.00e-04f  1
2963 -1.6076668e+03 1.05e-01 9.70e+01  -9.0 2.71e+02    -  6.40e-04 5.49e-03f  1
2964 -1.6080011e+03 1.05e-01 9.67e+01  -9.0 2.65e+02    -  8.71e-05 2.34e-03f  1
2965 -1.6118895e+03 1.05e-01 9.40e+01  -9.0 2.49e+02    -  8.63e-04 2.79e-02f  1
2966 -1.6173450e+03 1.13e-01 8.86e+01  -9.0 2.71e+02    -  1.47e-02 5.82e-02f  1
2967 -1.6178787e+03 1.13e-01 8.69e+01  -9.0 2.72e+02    -  6.07e-03 1.92e-02f  1
2968 -1.6186287e+03 1.18e-01 8.43e+01  -9.0 2.72e+02    -  8.37e-04 3.48e-02f  1
2969 -1.6201963e+03 2.11e-01 5.56e+03  -9.0 2.61e+02    -  3.73e-04 1.07e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2970 -1.6201963e+03 2.11e-01 5.56e+03  -9.0 2.77e+02    -  1.32e-01 4.00e-07h  1
2971 -1.6202457e+03 2.53e-01 4.91e+03  -9.0 2.93e+02    -  6.15e-03 7.17e-02f  1
2972r-1.6202457e+03 2.53e-01 9.99e+02  -0.6 0.00e+00  16.5 0.00e+00 9.75e-08R 21
2973r-1.6202457e+03 2.53e-01

ValueError: Cannot load a SolverResults object with bad status: error

In [ ]:
cnumber_range = range(1,57)

In [ ]:
def trans_cnumber(dic):
    molefraction = {}
    for i in range(1,57):
        molefraction[i] = []
    for i in m.COMP_ORG:
        molefraction[cal_cnumber(i)].append(np.array(dic[i]))
    for i in range(1,57):
        molefraction[i] = np.sum(molefraction[i],0)
    length = len(molefraction[1])
    tmp = {}
    for j in range(length):
        tmp[j] = []
        for i in range(1,57):
            tmp[j].append(molefraction[i][j])
    return tmp

In [ ]:
g_data = {}
d_data = {}
lr_data = {}
l1_data = {}
l2_data = {}
l3_data = {}

cd_x_data = {}
rf_x_data = {}

for t in Trange:

    g_data[t] = trans_cnumber(cd_data_master[t]['g'])
    d_data[t] = trans_cnumber(cd_data_master[t]['d'])
    lr_data[t] = trans_cnumber(cd_data_master[t]['d'])
    l1_data[t] = trans_cnumber(rf_data_master[t][1]['b'])
    l2_data[t] = trans_cnumber(rf_data_master[t][2]['b'])
    l3_data[t] = trans_cnumber(rf_data_master[t][3]['b'])
    
    cd_x_data[t] = trans_cnumber(cd_data_master[t]['x'])
    rf_x_data[t] = {}
    for j in model.reactive:
        rf_x_data[t][j] = trans_cnumber(rf_data_master[t][j]['x'])

In [ ]:
 def plot_distribution(index,temperature):
    fig, (ax,ax2) = plt.subplots(2,1,figsize=(16,12))
    ax.plot(cnumber_range,g_data[temperature][index],'co-')
    ax.plot(cnumber_range,d_data[temperature][index],'go-')
    ax.plot(cnumber_range,lr_data[temperature][index],'ro-',alpha=0.1)
    ax.plot(cnumber_range,l1_data[temperature][index],'ro-',alpha=0.3)
    ax.plot(cnumber_range,l2_data[temperature][index],'ro-',alpha=0.5)
    ax.plot(cnumber_range,l3_data[temperature][index],'ro-')
    ax.set_yscale("log")
    ax.set_ylim(1e-12, 1)
    ax.legend(['Vapor','Distillate','Reflux','L1','L2','Bottom'],fontsize=18)
    ax.set_title('T, Reflux {:.2f} kmol/s'.format(cd_data_master[temperature]['Re'][index]),fontsize=18)

    ax.set_ylabel('Molar Flow (kmol/s)', color='K',fontsize=18)
    ax.set_xlabel('Carbon Number', color='K',fontsize=18)
    # ax.tick_params('y', colors='k',labelsize=18)
    # ax.tick_params('x', colors='k',labelsize=18)

    ax2.plot(cnumber_range,cd_x_data[temperature][index],'go-')
    ax2.plot(cnumber_range,rf_x_data[temperature][1][index],'co-')
    ax2.plot(cnumber_range,rf_x_data[temperature][2][index],'bo-')
    ax2.plot(cnumber_range,rf_x_data[temperature][3][index],'ro-')

    ax2.set_ylim(0, 0.2)
    ax2.legend(['Condenser','Stage 1','Stage 2','Stage 3'],fontsize=18)
    ax2.set_title('Liquid Composition (Mole)',fontsize=18)

    ax2.set_ylabel('Molar Fraction', color='K',fontsize=18)
    ax2.set_xlabel('Carbon Number', color='K',fontsize=18)

    ax.grid()
    ax2.grid()
    plt.show()

In [ ]:
widgets.interact(plot_distribution, index = widgets.IntSlider(
    value=0,
    min=0,
    max=20,
    step=1,
    description='Reflux:'),temperature=widgets.SelectionSlider(
    options=Trange,
    value=Trange[0],
    description='Temperature',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True)
);

In [ ]:
plot_distribution(20,523.15)